In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession.builder\
        .master("spark://192.168.2.70:7077") \
        .appName("TianruZ_lecture2_shared_state")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max", 4)\
        .getOrCreate()


# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/12 18:40:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Immutable data for all workers
broadcast_variable = spark_context.broadcast({'apples': 100, 'oranges': 42, 'pears': 1})

In [3]:
rdd = spark_context.parallelize(['apples', 'pears'])

# Convert fruit to amounts of fruit.
# 'value' is captured in the closure, but its only a handle - the dictionary is serialized via a different route
result = rdd.map(lambda x: broadcast_variable.value[x])\
            .collect()

print(result)

[100, 1]


In [4]:
# ACCUMULATOR 

accumulator_1 = spark_context.accumulator(0)

rdd2 = spark_context.parallelize([1,2,3,4,5,6],2)



In [5]:
result = rdd2.foreach(lambda x: accumulator_1.add(1))

print(accumulator_1.value)

6


In [6]:
# release the cores for another application!
spark_context.stop()